In [1]:
from pybaseball import  playerid_lookup
from pybaseball import  statcast_pitcher
from pybaseball import  statcast
import pandas as pd
import numpy as np

In [19]:
df = statcast_pitcher('2023-04-01', '2023-09-30', 605483)
df.head()

Gathering Player Data


,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bat_speed,swing_length
0,CU,2023-09-25,82.7,1.91,6.47,"Snell, Blake",672275,605483,strikeout,swinging_strike_blocked,...,0,0,1,Standard,Standard,330.0,-0.077,-0.311,NaN,NaN
1,FF,2023-09-25,96.8,1.82,6.51,"Snell, Blake",672275,605483,NaN,swinging_strike,...,0,0,1,Standard,Standard,162.0,0.000,-0.098,NaN,NaN
2,FF,2023-09-25,96.8,1.92,6.54,"Snell, Blake",672275,605483,NaN,swinging_strike,...,0,0,1,Standard,Standard,124.0,0.000,-0.063,NaN,NaN
3,CH,2023-09-25,86.7,2.39,6.33,"Snell, Blake",571745,605483,single,hit_into_play,...,0,0,1,Standard,Standard,138.0,0.035,0.284,NaN,NaN
4,CU,2023-09-25,80.9,2.01,6.48,"Snell, Blake",571745,605483,NaN,ball,...,0,0,1,Standard,Standard,333.0,0.000,0.025,NaN,NaN


In [3]:
df.columns

Index(['pitch_type', 'game_date', 'release_speed', 'release_pos_x',
       'release_pos_z', 'player_name', 'batter', 'pitcher', 'events',
       'description', 'spin_dir', 'spin_rate_deprecated',
       'break_angle_deprecated', 'break_length_deprecated', 'zone', 'des',
       'game_type', 'stand', 'p_throws', 'home_team', 'away_team', 'type',
       'hit_location', 'bb_type', 'balls', 'strikes', 'game_year', 'pfx_x',
       'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b',
       'outs_when_up', 'inning', 'inning_topbot', 'hc_x', 'hc_y',
       'tfs_deprecated', 'tfs_zulu_deprecated', 'fielder_2', 'umpire', 'sv_id',
       'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot',
       'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed',
       'release_spin_rate', 'release_extension', 'game_pk', 'pitcher.1',
       'fielder_2.1', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6',
       'fielder_7', 'fielder_8', 'fielder_9', 'release_pos_y',
       'estima

In [20]:
df['pa_id'] = str(df['game_pk']) + "-" + str(df['at_bat_number'])

In [ ]:
df = df[

        'pa_id','pitch_type', 'player_name', 'batter', 'pitcher', 'events',
       'description', 'zone', 'des',
       'game_type', 'stand', 'p_throws', 'home_team', 'away_team', 'type',
       'hit_location', 'bb_type', 'balls', 'strikes', 'game_year', 'pfx_x',
       'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b',
       'outs_when_up', 'inning', 'inning_topbot', 'hc_x', 'hc_y',
       'tfs_deprecated', 'tfs_zulu_deprecated', 'fielder_2', 'umpire', 'sv_id',
       'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot',
       'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed',
       'release_spin_rate', 'release_extension', 'game_pk', 'pitcher.1',
       'fielder_2.1', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6',
       'fielder_7', 'fielder_8', 'fielder_9', 'release_pos_y',
       'estimated_ba_using_speedangle', 'estimated_woba_using_speedangle',
       'woba_value', 'woba_denom', 'babip_value', 'iso_value',
       'launch_speed_angle', 'at_bat_number', 'pitch_number', 'pitch_name',
       'home_score', 'away_score', 'bat_score', 'fld_score', 'post_away_score',
       'post_home_score', 'post_bat_score', 'post_fld_score',
       'if_fielding_alignment', 'of_fielding_alignment',
       'delta_home_win_exp', 'delta_run_exp', ]
     

In [21]:
pitch_dict = {'FF':0, 'FA':0,
              'FT':1, 'SI':1,
              'FC':2,
              'CU':3,'KC':3,'CS':3,'EP':3,
              'SL':4,
              'CH':5,'FS':5,'FO':5,'SC':5,
              'KN':6,
              'PO':np.nan}

# Map old pitch types to new mapping
df['pitch_type_map'] = df['pitch_type'].map(pitch_dict)
df.dropna(subset=['pitch_type_map'], inplace = True)


In [22]:
df['prev_pitch_1'] = df.groupby('pa_id')['pitch_type_map'].shift(1)
df['prev_pitch_2'] = df.groupby('pa_id')['pitch_type_map'].shift(2)
df['prev_pitch_3'] = df.groupby('pa_id')['pitch_type_map'].shift(3)


In [23]:
pitch_types = df['pitch_type'].unique()

# Function to calculate trailing 5-game proportions
def calculate_trailing_proportions(row):
    current_game = row['game_pk']
    
    # Get the last 5 unique games before the current row's game
    recent_games = df[df['game_pk'] < current_game]['game_pk'].drop_duplicates().tail(5)
    
    # Filter pitches from those recent games
    recent_pitches = df[df['game_pk'].isin(recent_games)]
    
    # Calculate the proportions for each pitch type
    proportions = {}
    total_pitches = len(recent_pitches)
    for pitch in pitch_types:
        count = recent_pitches[recent_pitches['pitch_type'] == pitch].shape[0]
        proportions[f'prop_{pitch}'] = count / total_pitches if total_pitches > 0 else 0
    
    return pd.Series(proportions)

# Apply the function row-wise to calculate trailing 5-game proportions
proportion_cols = df.apply(calculate_trailing_proportions, axis=1)
df = pd.concat([df, proportion_cols], axis=1)

# View the result
#df[['game_pk', 'pitch_type', ] + [f'prop_{pitch}' for pitch in pitch_types]]

#df[[f'prop_{pitch}' for pitch in pitch_types]] = df['pitch_type'].apply(calculate_trailing_proportions, axis=1)

df.tail()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,pa_id,pitch_type_map,pitch_type_copy,prev_pitch_1,prev_pitch_2,prev_pitch_3,prop_CU,prop_FF,prop_CH,prop_SL
3070,FF,2023-04-06,93.9,2.01,6.83,"Snell, Blake",621566,605483,field_out,hit_into_play,...,0 716441\n1 716441\n2 716441...,0.0,0.0,3.0,4.0,4.0,0.160643,0.568273,0.140562,0.130522
3071,FF,2023-04-06,93.1,1.92,6.92,"Snell, Blake",660670,605483,double,hit_into_play,...,0 716441\n1 716441\n2 716441...,0.0,0.0,0.0,3.0,4.0,0.160643,0.568273,0.140562,0.130522
3072,SL,2023-04-06,87.8,1.96,6.89,"Snell, Blake",660670,605483,NaN,ball,...,0 716441\n1 716441\n2 716441...,4.0,4.0,0.0,0.0,3.0,0.160643,0.568273,0.140562,0.130522
3073,FF,2023-04-06,94.7,2.05,6.90,"Snell, Blake",660670,605483,NaN,ball,...,0 716441\n1 716441\n2 716441...,0.0,0.0,4.0,0.0,0.0,0.160643,0.568273,0.140562,0.130522
3074,SL,2023-04-06,87.7,2.02,6.88,"Snell, Blake",660670,605483,NaN,swinging_strike,...,0 716441\n1 716441\n2 716441...,4.0,4.0,0.0,4.0,0.0,0.160643,0.568273,0.140562,0.130522


In [32]:
removal_list =  ['release_speed', 'release_pos_x',
            'release_pos_z', 'player_name', 'batter', 'pitcher', 'description',
            'zone', 'des', 'game_type', 'home_team',
            'away_team', 'type', 'game_year', 'pfx_x', 'pfx_z',
            'plate_x', 'plate_z', 'fielder_2', 'vx0', 'vy0', 'vz0', 'ax', 'ay',
            'az', 'sz_top', 'sz_bot', 'effective_speed', 'release_extension',
            'game_pk', 'pitcher.1', 'fielder_2.1', 'fielder_3', 'fielder_4',
            'fielder_5', 'fielder_6', 'fielder_7', 'fielder_8', 'fielder_9',
            'release_pos_y', 'pitch_name', 'game_date','post_away_score',
            'post_home_score', 'post_bat_score', 'post_fld_score'
            ,'on_3b', 'on_2b','on_1b'
            ,'hit_location', 'bb_type', 'events', 'spin_dir', 'spin_rate_deprecated', 'break_angle_deprecated', 'break_length_deprecated', 'woba_value', 'woba_denom', 'babip_value', 'iso_value'
            ,'tfs_deprecated', 'tfs_zulu_deprecated', 'umpire', 'sv_id'
            ,'hit_distance_sc', 'launch_speed', 'launch_angle', 'release_spin_rate'
            ,'hc_x', 'hc_y'
            ]

#dropping removal list from dataframe
df2 = df.drop(removal_list, axis=1)

In [30]:
df2.columns

Index(['pitch_type', 'stand', 'p_throws', 'balls', 'strikes', 'on_3b', 'on_2b',
       'on_1b', 'outs_when_up', 'inning', 'inning_topbot',
       'estimated_ba_using_speedangle', 'estimated_woba_using_speedangle',
       'launch_speed_angle', 'at_bat_number', 'pitch_number', 'home_score',
       'away_score', 'bat_score', 'fld_score', 'if_fielding_alignment',
       'of_fielding_alignment', 'spin_axis', 'delta_home_win_exp',
       'delta_run_exp', 'bat_speed', 'swing_length', 'pa_id', 'pitch_type_map',
       'pitch_type_copy', 'prev_pitch_1', 'prev_pitch_2', 'prev_pitch_3',
       'prop_CU', 'prop_FF', 'prop_CH', 'prop_SL'],
      dtype='object')

In [31]:
df2.head()

,pitch_type,stand,p_throws,balls,strikes,on_3b,on_2b,on_1b,outs_when_up,inning,...,pa_id,pitch_type_map,pitch_type_copy,prev_pitch_1,prev_pitch_2,prev_pitch_3,prop_CU,prop_FF,prop_CH,prop_SL
0,CU,R,L,0,2,NaN,682617.0,571745.0,2,6,...,0 716441\n1 716441\n2 716441...,3.0,3.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0
1,FF,R,L,0,1,NaN,682617.0,571745.0,2,6,...,0 716441\n1 716441\n2 716441...,0.0,0.0,3.0,NaN,NaN,0.0,0.0,0.0,0.0
2,FF,R,L,0,0,NaN,682617.0,571745.0,2,6,...,0 716441\n1 716441\n2 716441...,0.0,0.0,0.0,3.0,NaN,0.0,0.0,0.0,0.0
3,CH,R,L,1,0,NaN,NaN,682617.0,2,6,...,0 716441\n1 716441\n2 716441...,5.0,5.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
4,CU,R,L,0,0,NaN,NaN,682617.0,2,6,...,0 716441\n1 716441\n2 716441...,3.0,3.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0
